In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)

import bert
from bert.tokenization.bert_tokenization import FullTokenizer, printable_text
from keras_bert import load_trained_model_from_checkpoint

C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import re
import numpy as np

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

In [3]:
config_file = 'D:/GeekBrains/nlp/les13/bert_data/bert_config.json'
bert_model = 'D:/GeekBrains/nlp/les13/bert_data/bert_model.ckpt'
vocab_file = 'D:/GeekBrains/nlp/les13/bert_data/vocab.txt'

In [4]:
tokenizer = FullTokenizer(vocab_file=vocab_file, do_lower_case=False)
model = load_trained_model_from_checkpoint(config_file, bert_model, training=True)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 512)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
______________________________________________________________________________________________

### Предобученный берт для предсказания пропущенных слов

In [5]:
phrase = 'Данный [MASK] будет выдавать случайную [MASK].'

phrase = re.sub("\s*\[MASK\]\s*", "[MASK]", phrase)
print(phrase)

phrase = phrase.split('[MASK]')
tokens = ['[CLS]']

Данный[MASK]будет выдавать случайную[MASK].


In [6]:
for i in range(len(phrase)):
    if i == 0:
        tokens = tokens + tokenizer.tokenize(phrase[i]) 
    else:
        tokens = tokens + ['[MASK]'] + tokenizer.tokenize(phrase[i]) 
tokens = tokens + ['[SEP]']

In [7]:
tokens

['[CLS]',
 'Да',
 '##нный',
 '[MASK]',
 'будет',
 'вы',
 '##дава',
 '##ть',
 'случай',
 '##ную',
 '[MASK]',
 '.',
 '[SEP]']

In [8]:
def token_converter(tokens):
    token_input = tokenizer.convert_tokens_to_ids(tokens) 
    token_input = token_input + [0] * (512 - len(token_input))

    mask_input = [0]*512
    for i in range(len(mask_input)):
        if token_input[i] == 103:
            mask_input[i] = 1
    seg_input = [0]*512

    token_input = np.asarray([token_input])
    mask_input = np.asarray([mask_input])
    seg_input = np.asarray([seg_input])
    
    return token_input, mask_input, seg_input

token_input, mask_input, seg_input = token_converter(tokens)

predicts = model.predict([token_input, seg_input, mask_input])[0] 
predicts = np.argmax(predicts, axis=-1)
predicts = predicts[0][:len(tokens)]

In [9]:
out = []
for i in range(len(mask_input[0])):
    if mask_input[0][i] == 1:                       
        out.append(predicts[i]) 

out = tokenizer.convert_ids_to_tokens(out) 
print(out)
out = ' '.join(out)                               
out = printable_text(out)            
out = out.replace(' ##','') 
out

['граф', 'функции']


'граф функции'

### Предобученный берт для предсказания последовательности двух фраз

In [10]:
tokens_sen_1 = tokenizer.tokenize("Я пошел в магазин")
tokens_sen_2 = tokenizer.tokenize("И купил там молоко")

tokens = ['[CLS]'] + tokens_sen_1 + ['[SEP]'] + tokens_sen_2 + ['[SEP]']

In [11]:
tokens

['[CLS]',
 'Я',
 'по',
 '##шел',
 'в',
 'магазин',
 '[SEP]',
 'И',
 'к',
 '##уп',
 '##ил',
 'там',
 'м',
 '##оло',
 '##ко',
 '[SEP]']

In [12]:
token_input, mask_input, seg_input = token_converter(tokens)

In [13]:
predicts = model.predict([token_input, seg_input, mask_input])[1] 
predicts[0][0]

0.9489631

### Fine-tuning bert

In [14]:
df_train = pd.read_csv("D:/GeekBrains/nlp/les13/lesson_13/data/train.csv")
df_val = pd.read_csv("D:/GeekBrains/nlp/les13/lesson_13/data/val.csv")

In [15]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [16]:
train_tokens = map(tokenizer.tokenize, df_train['text'])
train_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], train_tokens)
train_token_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
train_token_ids = map(lambda tids: tids + [0] * (150 - len(tids)), train_token_ids) #141 - max len of train_token_ids
train_token_ids = np.vstack([np.array(x) for x in list(train_token_ids)])

test_tokens = map(tokenizer.tokenize, df_val['text'])
test_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], test_tokens)
test_token_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

test_token_ids = map(lambda tids: tids + [0] * (150 - len(tids)), test_token_ids)
test_token_ids = np.vstack([np.array(x) for x in list(test_token_ids)])

train_labels_final = df_train['class'].values
test_labels_final = df_val['class'].values

In [17]:
# max(list(map(lambda tids: len(tids), test_token_ids)))

In [18]:
# train_token_ids

In [19]:
bert_params = bert.params_from_pretrained_ckpt('D:/GeekBrains/nlp/les13/bert_data/')
bert_layer = bert.BertModelLayer.from_params(bert_params)

    
model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(150,), dtype='int32'),
        bert_layer,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])

model.build(input_shape=(None, 150))
bert_layer.apply_adapter_freeze()
bert_layer.trainable = False
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(lr=0.00001), metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert_model_layer (BertModelL (None, 150, 768)          177261312 
_________________________________________________________________
flatten (Flatten)            (None, 115200)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                7372864   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 184,634,241
Trainable params: 7,372,929
Non-trainable params: 177,261,312
_________________________________________________________________
None


C:\Users\snetkova\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [20]:
history = model.fit(
        train_token_ids,
        train_labels_final,
        epochs=5,
        validation_data=(test_token_ids, test_labels_final),
        verbose=1
    )

Epoch 1/5
5671/5671 [==============================] - 5893s 1s/step - loss: 6.0478e-08 - accuracy: 0.4961 - val_loss: 6.0170e-08 - val_accuracy: 0.4953
Epoch 2/5
5671/5671 [==============================] - 5889s 1s/step - loss: 6.0478e-08 - accuracy: 0.4973 - val_loss: 6.0170e-08 - val_accuracy: 0.4953
Epoch 3/5
5671/5671 [==============================] - 5881s 1s/step - loss: 6.0478e-08 - accuracy: 0.4968 - val_loss: 6.0170e-08 - val_accuracy: 0.4953
Epoch 4/5
5671/5671 [==============================] - 5901s 1s/step - loss: 6.0478e-08 - accuracy: 0.4970 - val_loss: 6.0170e-08 - val_accuracy: 0.4953
Epoch 5/5
5671/5671 [==============================] - 5903s 1s/step - loss: 6.0478e-08 - accuracy: 0.4963 - val_loss: 6.0170e-08 - val_accuracy: 0.4953


In [22]:
model.save('./saved_model/mdl')

INFO:tensorflow:Assets written to: ./saved_model/mdl\assets


INFO:tensorflow:Assets written to: ./saved_model/mdl\assets


In [24]:
model.evaluate(test_token_ids, test_labels_final, batch_size=64, verbose=1)[1]

355/355 [==============================] - 604s 2s/step - loss: 6.0170e-08 - accuracy: 0.4953


0.4952607750892639

Модель не обучилась